## Load scores tensor (pretrained)

In [1]:
import pandas as pd
import pickle
import pandas as pd
model_name = 'SimpleX'
data_name = 'ml-1m'

# scores_tensor_df = pd.read_csv(f'./pretrained_scores/{data_name}/{model_name}_{data_name}_scores_tensor.csv', header=None)
scores_tensor_df = pd.read_csv(f'./pretrained_scores/{data_name}/{model_name}_{data_name}_scores_tensor.txt', header=None)
scores_tensor_df

,0,1,2,3,4,5,6,7,8,9,...,844,845,846,847,848,849,850,851,852,853
0,0.331406,0.280178,-1.000000e+08,-1.000000e+08,5.195839e-01,-1.000000e+08,0.402708,4.867731e-01,0.474337,0.422230,...,0.630990,0.333921,-1.000000e+08,-1.000000e+08,0.073853,0.651333,0.438040,0.441256,0.334835,0.360692
1,0.290129,0.304653,4.168867e-01,5.879234e-01,4.549532e-01,4.588683e-01,0.399067,5.706309e-01,0.446806,0.424417,...,0.546390,0.314037,6.225903e-01,5.334495e-01,0.098213,0.589729,0.393379,0.410857,0.329335,0.430018
2,0.345323,0.210515,-1.000000e+08,5.592080e-01,-1.000000e+08,6.397491e-01,0.388027,-1.000000e+08,0.482791,0.393186,...,0.493282,0.323248,4.366275e-01,4.932338e-01,0.139200,0.414490,0.524191,0.377856,0.373770,0.383036
3,0.335566,0.313333,3.918142e-01,5.505714e-01,4.457762e-01,4.060564e-01,0.414192,4.799969e-01,0.469270,0.398371,...,0.443196,0.268232,4.646194e-01,4.701371e-01,0.149726,0.468727,0.362499,0.419989,0.346085,0.438212
4,0.316109,0.324808,4.062611e-01,5.064892e-01,4.199317e-01,3.910332e-01,0.390788,4.642188e-01,0.458737,0.393376,...,0.421836,0.189362,4.275019e-01,4.826458e-01,0.191160,0.449492,0.363881,0.375864,0.323614,0.416821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.420022,0.444244,4.852486e-01,4.730469e-01,3.523092e-01,5.205200e-01,0.268604,4.395292e-01,0.296861,0.482562,...,0.341211,0.481065,4.170464e-01,4.659184e-01,0.303879,0.336407,0.566173,0.279829,0.479844,0.526327
1506,0.384294,0.398105,4.601306e-01,5.583086e-01,4.251068e-01,5.981378e-01,0.345325,4.813348e-01,0.396453,0.416362,...,0.491878,0.343988,5.097014e-01,5.010271e-01,0.360509,0.464977,0.508359,0.402585,0.402791,0.453156
1507,0.199680,0.197117,5.266573e-01,5.236182e-01,3.681665e-01,3.835050e-01,0.416929,-1.000000e+08,0.384005,0.368110,...,0.439530,0.369540,4.629174e-01,4.976031e-01,0.004933,0.393458,0.424172,0.382394,0.330114,0.344768
1508,0.260489,0.245708,4.592009e-01,5.049434e-01,4.797631e-01,3.706224e-01,0.489580,5.669957e-01,0.423839,0.427352,...,0.659947,0.372320,7.461454e-01,4.634248e-01,-0.074940,0.657177,0.381077,0.379598,0.297279,0.355750


## ItemRec

In [17]:
import torch
from collections import defaultdict
import pandas as pd
import numpy as np
from itertools import chain, repeat

limit_num = 50
topk = 10
item_limit_num = 5
# scores_tensor_df = scores_tensor_df.iloc[:,:]

scores_tensor = torch.tensor(scores_tensor_df.to_numpy())
scores_sorted, idx_sorted = torch.sort(scores_tensor, dim=-1, descending=True)

scores_tensor = scores_sorted[:,:limit_num]
scores_tensor_user = idx_sorted[:,:limit_num]


item_list = scores_tensor_user.flatten()
user_list = np.tile(np.arange(scores_tensor.shape[0]).reshape(-1,1),limit_num).flatten()
scores_list = scores_tensor.flatten()
idx_item_list = np.tile(np.arange(limit_num).reshape(1,-1), (scores_tensor.shape[0],1)).flatten()
user_item_df = pd.DataFrame({'uid':user_list, 'iid':item_list, 'idx_item':idx_item_list, 'score':scores_list})

# general top-k recommendation results
top_k_item_origin = user_item_df.sort_values(by=['uid', 'score'],ascending=[True, False]).groupby('uid',sort=False).head(topk).loc[:,['uid','iid','score']]


# get maximum and minimum score of each item
user_item_df = user_item_df.sort_values(by='iid')
item_norm_scores_g = user_item_df.groupby(by='iid',sort=False)
item_scores_max, item_scores_min = item_norm_scores_g.max()['score'].tolist(), item_norm_scores_g.min()['score'].tolist()
uid_list_len = item_norm_scores_g.count().iloc[:,0].tolist()

user_item_df['min_score'] = list(chain(*[list(repeat(s,uid_len)) for s, uid_len in zip(item_scores_min, uid_list_len)]))
user_item_df['max_score'] = list(chain(*[list(repeat(s,uid_len)) for s, uid_len in zip(item_scores_max, uid_list_len)]))


# item's rank in user
user_item_df['rank_item'] = user_item_df['idx_item'] - (user_item_df['score'] - user_item_df['min_score']) / (user_item_df['max_score'] - user_item_df['min_score'])

# user's rank in item
user_item_df = user_item_df.sort_values(by=['iid','rank_item'])
top_kitem_user = user_item_df.groupby('iid',sort=False).head(item_limit_num).loc[:,['uid','iid','score']]

# trans to user-item

uid_idx_ = list(chain(*[list(range(u_len)) for u_len in uid_list_len]))
uid_len_ = list(chain(*[list(repeat(uid_len,uid_len)) for uid_len in uid_list_len]))
user_item_df['uid_len'] = uid_len_
user_item_df['idx_user'] = uid_idx_
user_item_df['rank_user'] = user_item_df['idx_user'] + 1/user_item_df['uid_len']


# user-item sort
user_item_df = user_item_df.sort_values(by=['uid','rank_user'])
top_k_item = user_item_df.groupby('uid',sort=False).head(topk).loc[:,['uid','iid','score']]

# top_k_item
final_df = pd.merge(left=top_k_item, right=top_kitem_user, left_on=['uid','iid'], right_on=['uid','iid'],how='inner')
final_df = pd.merge(left=final_df, right=top_k_item_origin, left_on=['uid','iid'], right_on=['uid','iid'], how='outer')
final_df = final_df.groupby(by='uid',sort=False).head(10).sort_values(by=['uid','score'],ascending=[True,False])
topk_idx = torch.tensor(final_df['iid'].tolist()).cuda().reshape(scores_tensor.shape[0], topk)
topk_idx2 = torch.tensor(top_k_item_origin['iid'].tolist()).cuda().reshape(scores_tensor.shape[0], topk)

## Load test data

In [18]:
import torch
import pickle
from collections import defaultdict

# Load data
path = f'./datasets/{data_name}/{data_name}-dataset_test.txt'
test_df = pd.read_csv(path, header=None)
test_user_list = defaultdict(list)
for i,x in test_df.iterrows():
    test_user_list[x[0]].append(x[1])

## Valid Coverage

In [19]:
valid_coverage = set()
valid_coverage_topk2 = set()
for uid in range(topk_idx.shape[0]):
    valid_coverage = valid_coverage | set(topk_idx[uid].tolist()) & set(test_user_list[uid])
    valid_coverage_topk2 = valid_coverage_topk2 | set(topk_idx2[uid].tolist()) & set(test_user_list[uid])


item_num = scores_tensor_df.shape[1]

valid_coverage = len(valid_coverage) / item_num
valid_coverage_topk2 = len(valid_coverage_topk2) / item_num
valid_coverage, valid_coverage_topk2

(0.2939110070257611, 0.25878220140515223)

## NDCG

In [7]:
pos_len = []
for u in range(topk_idx.shape[0]):
    pos_len.append(len(test_user_list[u]))

pos_index = np.zeros(topk_idx.shape)
pos_index2 = np.zeros(topk_idx2.shape)

for u in range(topk_idx.shape[0]):
    for j in range(topk):
        if topk_idx[u,j].item() in test_user_list[u]:
            pos_index[u,j] = 1
        
        if topk_idx2[u,j].item() in test_user_list[u]:
            pos_index2[u,j] = 1


len_rank = np.full_like(pos_len, pos_index.shape[1])
idcg_len = np.where(pos_len > len_rank, len_rank, pos_len)
iranks = np.zeros_like(pos_index, dtype=np.float32)
iranks[:, :] = np.arange(1, pos_index.shape[1] + 1)
idcg = np.cumsum(1.0 / np.log2(iranks + 1), axis=1)
for row, idx in enumerate(idcg_len):
    idcg[row, idx:] = idcg[row, idx - 1]

ranks = np.zeros_like(pos_index, dtype=np.float32)
ranks[:, :] = np.arange(1, pos_index.shape[1] + 1)
dcg = 1.0 / np.log2(ranks + 1)
dcg = np.cumsum(np.where(pos_index, dcg, 0), axis=1)
ndcg = dcg / idcg


iranks2 = np.zeros_like(pos_index2, dtype=np.float32)
iranks2[:, :] = np.arange(1, pos_index2.shape[1] + 1)
idcg2 = np.cumsum(1.0 / np.log2(iranks2 + 1), axis=1)
for row, idx in enumerate(idcg_len):
    idcg2[row, idx:] = idcg2[row, idx - 1]

ranks2 = np.zeros_like(pos_index2, dtype=np.float32)
ranks2[:, :] = np.arange(1, pos_index2.shape[1] + 1)
dcg2 = 1.0 / np.log2(ranks2 + 1)
dcg2 = np.cumsum(np.where(pos_index2, dcg2, 0), axis=1)
ndcg2 = dcg2 / idcg2

np.mean(ndcg,axis=0)[-1], np.mean(ndcg2,axis=0)[-1]

(0.08647759, 0.09061509)

## Gini

In [8]:
cnt = [0 for i in range(item_num)]
row, col = topk_idx.shape
for i in range(row):
    for j in range(col):
        cnt[topk_idx[i,j].item()] += 1

cnt.sort()
giny = 0
height, area = 0, 0
for c in cnt:
    height += c
    area += height-c/2
fair_area = height*item_num/2
giny = (fair_area-area)/fair_area
giny

0.7508235494827613